In [28]:
import os, sys
sys.path.append(os.getcwd())
sys.modules['theano'] = None

import numpy as np
import tensorflow as tf
import scipy.misc
import imageio
from imageio import imwrite
from scipy.misc import imsave, imread

import tflib as lib
import tflib.sampling_loop
import tflib.ops.kl_unit_gaussian
import tflib.ops.kl_gaussian_gaussian
import tflib.ops.conv2d
import tflib.ops.linear
import tflib.ops.batchnorm
import tflib.ops.embedding

import os
import urllib
import gzip
import cPickle as pickle

import time
import functools
import random

In [29]:
from fuel.datasets.hdf5 import H5PYDataset
from fuel.schemes import ShuffledScheme, SequentialScheme
from fuel.streams import DataStream
# from fuel.transformers.image import RandomFixedSizeCrop

PATH = '/Users/wildflowerlyi/Desktop/Image_Samples/cifar10.hdf5'

In [30]:
import h5py
import keras
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape

(50000, 32, 32, 3)

In [31]:
x_train_new = x_train.transpose(0,3,1,2)
x_train_new.shape

(50000, 3, 32, 32)

In [14]:
def color_grid_vis(X, nh, nw, savepath):
    # from github.com/Newmu
    X = X.transpose(0,2,3,1)
    h, w = X[0].shape[:2]
    img = np.zeros((h*nh, w*nw, 3))
    for n, x in enumerate(X):
        j = n/nw
        i = n%nw
        img[j*h:j*h+h, i*w:i*w+w, :] = x
    imsave(savepath,img)

In [35]:
#x_train_array = np.array(x_train_new)
image = x_train_new[1,:]
print(image.shape)
image_array = image.reshape(-1,3,32,32)
print(image_array.shape)
x_array = np.array(image_array)
x_array.shape

(3, 32, 32)
(1, 3, 32, 32)


(1, 3, 32, 32)

In [33]:
color_grid_vis(image_array,1,1,'/Users/wildflowerlyi/Desktop/scale.png')

/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # Remove the CWD from sys.path while we load stuff.


In [4]:
def _make_stream(stream, bs):
    def new_stream():
      result = np.empty((bs, 32, 32, 3), dtype='int32')
      for (imb,) in stream.get_epoch_iterator():
        for i, img in enumerate(imb):
          result[i] =  img[:32, :32, :]                
        yield (result.transpose(0,3,1,2),)
    return new_stream

In [5]:
def load(batch_size=128): #LEILAEDIT: took out downsampling, changed validation data to CIFAR test set, changed tr_data in line 53 to val_data
    
    tr_data = H5PYDataset(PATH, which_sets=('train',))
    val_data = H5PYDataset(PATH, which_sets=('test',))

    ntrain = tr_data.num_examples
    nval = val_data.num_examples

    print "ntrain {}, nval {}".format(ntrain, nval)

    tr_scheme = ShuffledScheme(examples=ntrain, batch_size=batch_size)
    tr_stream = DataStream(tr_data, iteration_scheme=tr_scheme)

    # te_scheme = SequentialScheme(examples=ntest, batch_size=batch_size)
    # te_stream = DataStream(te_data, iteration_scheme=te_scheme)

    val_scheme = SequentialScheme(examples=nval, batch_size=batch_size)
    val_stream = DataStream(val_data, iteration_scheme=val_scheme)

    return _make_stream(tr_stream, batch_size), _make_stream(val_stream, batch_size)

In [6]:
data1, data2 = load()

ntrain 50000, nval 10000


In [7]:
import h5py
data0 = h5py.File(PATH,'r')
data0.keys()

[u'features', u'targets']

In [8]:
featurearray = np.array(data0['features'][:])
featurearray.shape
firstimage = featurearray[1,:]
firstimage.shape
firstimage2 = firstimage.reshape(1, 3, 32, 32)
firstimage2.shape
firstimage3 = np.array(firstimage.reshape(3*32*32))
firstimage3.shape

(3072,)

In [9]:
color_grid_vis(firstimage2, 1, 1, '/Users/wildflowerlyi/Desktop/testimage.png')

/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  # Remove the CWD from sys.path while we load stuff.


In [10]:
def loadsimple(batch_size=128): #LEILAEDIT: took out downsampling, changed validation data to CIFAR test set, changed tr_data in line 53 to val_data
    
    tr_data = H5PYDataset(PATH, which_sets=('train',))
    val_data = H5PYDataset(PATH, which_sets=('test',))

    ntrain = tr_data.num_examples
    nval = val_data.num_examples

    print "ntrain {}, nval {}".format(ntrain, nval)

    tr_scheme = ShuffledScheme(examples=ntrain, batch_size=batch_size)
    tr_stream = DataStream(tr_data, iteration_scheme=tr_scheme)

    # te_scheme = SequentialScheme(examples=ntest, batch_size=batch_size)
    # te_stream = DataStream(te_data, iteration_scheme=te_scheme)

    val_scheme = SequentialScheme(examples=nval, batch_size=batch_size)
    val_stream = DataStream(val_data, iteration_scheme=val_scheme)

    return tr_stream, val_stream

In [11]:
datasimple1, datasimple2 = loadsimple()

ntrain 50000, nval 10000
